In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, f1_score, fbeta_score, precision_score, recall_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import ParameterGrid
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm.notebook import tqdm
import csv
from gensim.models.callbacks import CallbackAny2Vec
from itertools import groupby
from gensim.models import Word2Vec
import gensim
from pathlib import Path
import matplotlib.pyplot as plt
pd.options.display.max_columns = 999
import fasttext
from sklearn.model_selection import train_test_split

In [490]:
train = pd.read_csv('train.csv', delimiter=',')

In [491]:
test= pd.read_csv('test.csv', delimiter=',')

In [492]:
train.loc[train['keyword'].notnull(), 'text'] = train['text'] + '. ' + train['keyword']
test.loc[test['keyword'].notnull(), 'text'] = test['text'] + '. ' + test['keyword']

In [493]:
all_twitts=[]
for twitt in train.text:
    this_twitt=str()
    for word in compiler.findall(twitt.lower()):
        this_twitt+=word
        this_twitt+=' '
    all_twitts.append(this_twitt)

In [494]:
train['text']=all_twitts

In [495]:
all_twitts=[]
for twitt in test.text:
    this_twitt=str()
    for word in compiler.findall(twitt.lower()):
        this_twitt+=word
        this_twitt+=' '
    all_twitts.append(this_twitt)

In [496]:
test['text']=all_twitts

In [497]:
import string
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# NLTK Tweet Tokenizer for now
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True)

corpus = []

def clean_text(text):
    text = re.sub(r"\x89Û_", "", text)
    text = re.sub(r"\x89ÛÒ", "", text)
    text = re.sub(r"\x89ÛÓ", "", text)
    text = re.sub(r"\x89ÛÏWhen", "When", text)
    text = re.sub(r"\x89ÛÏ", "", text)
    text = re.sub(r"China\x89Ûªs", "China's", text)
    text = re.sub(r"let\x89Ûªs", "let's", text)
    text = re.sub(r"\x89Û÷", "", text)
    text = re.sub(r"\x89Ûª", "", text)
    text = re.sub(r"\x89Û\x9d", "", text)
    text = re.sub(r"å_", "", text)
    text = re.sub(r"\x89Û¢", "", text)
    text = re.sub(r"\x89Û¢åÊ", "", text)
    text = re.sub(r"fromåÊwounds", "from wounds", text)
    text = re.sub(r"åÊ", "", text)
    text = re.sub(r"åÈ", "", text)
    text = re.sub(r"JapÌ_n", "Japan", text)    
    text = re.sub(r"Ì©", "e", text)
    text = re.sub(r"å¨", "", text)
    text = re.sub(r"SuruÌ¤", "Suruc", text)
    text = re.sub(r"åÇ", "", text)
    text = re.sub(r"å£3million", "3 million", text)
    text = re.sub(r"åÀ", "", text)
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r'[0-9]', '', text)
    
    for p in string.punctuation.replace('!', ''):
        text = text.replace(p, '')
        
    text = re.sub(r'http\S+', '', text)
    
   # text = [w.lower() for w in text if not w in stop_words]
   # corpus.append(text)
    
  #  text = ' '.join(text)
    
    return text

In [498]:
train['text'] = train['text'].apply(lambda s: clean_text(s))
test['text'] = test['text'].apply(lambda s: clean_text(s))

# see some cleaned data
train.sample(5)

,id,keyword,location,text,target
6675,9566,thunder,"Decatur, GA",kristyleemusic brings her alabama thunder back...,0
3186,4575,emergency%20plan,"Atlanta, GA",bc it costs less to have sick people using eme...,1
1161,1672,bridge%20collapse,Mumbai,ameenshaikh by ur logic if bridge didnt collap...,1
6672,9563,thunder,"South Carolina, USA",the worst part is seeing lightning and trying ...,0
5458,7785,police,NaN,photo lastma officials challenge police for ...,1


In [499]:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
train.loc[train['id'].isin(ids_with_target_error),'target'] = 0

In [500]:
import re
compiler = re.compile("[\w'#:/.]+")

In [501]:
def create_file(x, y, fname):
    with open(fname, 'w', encoding='utf-8') as f:
        for i, x_i in enumerate(x):
            if i > 0: f.write('\n')
            f.write(f'__label__{y[i]} {x_i}')

In [502]:
def predict(model, X):
    probs=[]
    for labels, scores in zip(*model.predict(X, k=2)):
        probs.append(dict(zip(labels, scores))['__label__1'])
    return probs

In [503]:
X_train, X_test, y_train, y_test = train_test_split(train['text'], train['target'], test_size=0.25, random_state=42)

In [504]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

In [505]:
create_file(X_train, y_train.to_list(), 'train.data')
create_file(X_test, y_test.to_list(), 'test.data')

In [506]:
model_ftext = fasttext.train_supervised(input='train.data',  minCount=3, lr=0.1, wordNgrams=1, minn=9, maxn=9, dim=100, epoch=5, loss='softmax')

In [507]:
from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score
import numpy as np

print('--CLASS 1--')
print('Training ROC-AUC: ', round(roc_auc_score(y_train, predict(model_ftext, list(X_train))), 2))
print('Testing ROC-AUC', round(roc_auc_score(y_test, predict(model_ftext, list(X_test))), 2))
print('Training PR-AUC', round(average_precision_score(y_train, predict(model_ftext, list(X_train))),2))
print('Testing PR-AUC', round(average_precision_score(y_test, predict(model_ftext, list(X_test))),2))

--CLASS 1--
Training ROC-AUC:  0.94
Testing ROC-AUC 0.86
Training PR-AUC 0.93
Testing PR-AUC 0.85


In [508]:
see_preds = pd.DataFrame()
see_preds['twitt'] = X_test
see_preds['true'] = y_test
see_preds['proba'] = predict(model_ftext, list(X_test))
see_preds['prediction'] = see_preds['proba'].apply(lambda x: 1 if x>0.5 else 0)
see_preds['correct'] = [1 if x==y else 0 for x, y in zip(see_preds['prediction'], see_preds['true'])]
print('Accuracy: ', round(see_preds.correct.mean()*100, 2), '%')
see_preds.head(3)

Accuracy:  80.83 %


,twitt,true,proba,prediction,correct
0,so you have a new weapon that can cause un ima...,1,0.016756,0,0
1,the f amp ing things i do for gishwhes just go...,0,0.157940,0,1
2,dt georgegalloway rt gallowaymayor ûïthe col p...,1,0.523016,1,1


In [520]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def evaluate_model(params):
    model= fasttext.train_supervised(input='train.data', 
                                     minCount=params['minCount'], 
                                     lr=params['lr'], 
                                     wordNgrams=params['wordNgrams'], 
                                     minn=params['minn'], 
                                     maxn=params['maxn'], 
                                     dim=params['dim'], 
                                     epoch=params['epoch'], 
                                     loss=params['loss_func'])
    
    see_preds = pd.DataFrame()
    see_preds['twitt'] = X_test
    see_preds['true'] = y_test
    see_preds['proba'] = predict(model, list(X_test))
    see_preds['prediction'] = see_preds['proba'].apply(lambda x: 1 if x>0.5 else 0)
    see_preds['correct'] = [1 if x==y else 0 for x, y in zip(see_preds['prediction'], see_preds['true'])]
    accuracy = round(see_preds.correct.mean()*100, 3)

    return {
        'minCount': params['minCount'],
        'lr': params['lr'], 
        'wordNgrams': params['wordNgrams'], 
        'dim': params['dim'],
        'epoch': params['epoch'],
        'loss_func': params['loss_func'],
        'minn': params['minn'],
        'maxn': params['maxn'],
        'Training ROC-AUC': round(roc_auc_score(y_train, predict(model, list(X_train))), 3),
        'Testing ROC-AUC': round(roc_auc_score(y_test, predict(model, list(X_test))), 3),
        'Training PR-AUC': round(average_precision_score(y_train, predict(model, list(X_train))),3),
        'Testing PR-AUC': round(average_precision_score(y_test, predict(model, list(X_test))), 3),
        'Testing ACCURACY': accuracy
    }

def objective(params):
    res = evaluate_model(params)
    
    res['loss'] = - res['Testing ACCURACY'] # Esta loss es la que hyperopt intenta minimizar
    res['status'] = STATUS_OK # Asi le decimos a hyperopt que el experimento salio bien
    return res 

# Este espacio es una copia del grid search, no estamos dando mucha información en este prior
hyperparameter_space = {
        'minCount': hp.choice('minCount', range(1, 10)),
        'lr': hp.uniform('lr',0.0001, 0.3),
        'wordNgrams': hp.choice('wordNgrams', range(1, 4)),
        'dim': hp.choice('dim', range(5, 300)),
        'epoch': hp.choice('epoch', range(2, 25)),
        'loss_func': hp.choice('loss_func', ['hs', 'softmax', 'ns']),
        'minn': hp.choice('minn', range(2, 5)),
        'maxn': hp.choice('maxn', range(4,8))
}

In [521]:
trials = Trials()
fmin(
    objective,
    space=hyperparameter_space,
    algo=tpe.suggest,
    max_evals=200,
    trials=trials
);

100%|█████████████████████████████████████████████████████████| 200/200 [13:43<00:00,  4.12s/trial, best loss: -81.723]


In [578]:
experiment_results = pd.DataFrame(trials.results)
experiment_results.sort_values(by='Training ROC-AUC').head(45)

,minCount,lr,wordNgrams,dim,epoch,loss_func,minn,maxn,Training ROC-AUC,Testing ROC-AUC,Training PR-AUC,Testing PR-AUC,Testing ACCURACY,loss,status
1,2,0.014809,2,213,10,ns,2,7,0.598,0.604,0.487,0.487,57.458,-57.458,ok
40,6,0.006710,2,175,11,ns,4,4,0.611,0.625,0.506,0.519,57.458,-57.458,ok
173,5,0.009138,3,54,3,softmax,2,4,0.632,0.636,0.538,0.546,61.870,-61.870,ok
44,8,0.075374,2,142,4,ns,4,7,0.684,0.661,0.576,0.550,57.458,-57.458,ok
45,4,0.019106,1,257,2,softmax,2,4,0.692,0.698,0.598,0.603,45.116,-45.116,ok
194,7,0.026422,1,78,5,softmax,2,5,0.706,0.708,0.623,0.630,57.511,-57.511,ok
32,4,0.004146,2,122,20,hs,4,5,0.718,0.700,0.650,0.624,51.155,-51.155,ok
168,2,0.045648,3,157,2,softmax,3,4,0.727,0.713,0.663,0.647,66.912,-66.912,ok
3,8,0.028704,3,246,8,hs,2,7,0.745,0.737,0.683,0.666,57.773,-57.773,ok
6,6,0.026176,1,74,16,hs,2,7,0.746,0.738,0.693,0.671,57.458,-57.458,ok


Train with all data with those hyperparameters

In [528]:
create_file(train['text'], train['target'].to_list(), 'final_train.data')
#create_file(X_test, y_test.to_list(), 'test.data')

In [579]:
model_ftext = fasttext.train_supervised(input='final_train.data',  minCount=1, lr=0.229457, wordNgrams=2, minn=4, maxn=6, dim=68, epoch=5, loss='hs')

In [580]:
from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score
import numpy as np

print('--CLASS 1--')
print('Training ROC-AUC: ', round(roc_auc_score(train['target'].to_list(), predict(model_ftext, list(train['text']))), 2))
print('Training PR-AUC', round(average_precision_score(train['target'].to_list(), predict(model_ftext, list(train['text']))),2))

--CLASS 1--
Training ROC-AUC:  0.91
Training PR-AUC 0.9


In [581]:
preds = predict(model_ftext, list(test['text']))

In [582]:
sample = pd.read_csv('sample_submission.csv')

In [583]:
submission=pd.DataFrame()
submission['id'] = test['id']
submission['preds'] = preds

In [584]:
submission['target'] = submission['preds'].apply(lambda x: 1 if x>0.5 else 0)

In [585]:
submission.drop('preds', axis=1, inplace=True)

In [586]:
len(submission)

3263

In [587]:
submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,1
4,11,1


In [588]:
submission.to_csv('Submission4.csv', index=False)